In [153]:
import tracemalloc # can benchmark the memory usage of the program.
import time # can benchmark the time usage for the program


In [252]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)


def hirschberg(v, w, delta, mismatch):

  if len(v) == 0: # base cases
    vout = "-" * len(w)
    wout = w
    score = -len(w)
    return vout, wout, score

  if len(w) == 0:
    vout = v
    wout = "-" * len(v)
    score = -len(v)
    return vout, wout, score

  if len(v) == 1 and len(w) == 1:
    # the midpoint could land on the endpoint.
    # in this case, the midpoint will always land on one of the endpoints
    # but also, it's simple to analyze: either matches in which case we get +1 or
    # or it doesn't match in which case we can either take the diagonal

    if v == w:
      return v, w, 1 # match
    else:
        return v, w, -1

  height, width = len(v) + 1, len(w) + 1 # number of cells. so cell 0 lies before letter 0
  mid_point = width // 2

  ## forward
  fwd = [i for i in range(0, -height, -1)] # initialize with descending integers
  for i in range(1, mid_point + 1):
    nxt = [0] * height
    nxt[0] = fwd[0] - 1
    for j in range(1, height):
      nxt[j] = max(
          fwd[j] - 1,
          nxt[j-1] - 1,
          fwd[j-1] + delta[v[j-1]][w[i-1]]
          )

    fwd, nxt = nxt, [0] * height

  ## backward
  bwd = [i for i in range(-height + 1, 1)] # initialize with ascending integers
  for i in range(width-2, mid_point-1, -1):
    nxt = [0] * height
    nxt[-1] = bwd[-1] - 1
    for j in range(height-1 -1, -1, -1):

      nxt[j] = max(
          bwd[j] - 1,
          nxt[j+1] - 1,
          bwd[j+1] + delta[v[j]][w[i]]
          )

    bwd, nxt = nxt, [0] * height

  ## find i_star
  i_star = -1
  largest = float("-inf")
  mid = [fwd[i] + bwd[i] for i in range(len(fwd))]
  for i, m in enumerate(mid):

    if m > largest:
      i_star = i
      largest = m

  if mid_point == len(w) and i_star == len(v):
    # there are two points to test. if it's a match you just do diagonal
    # otherwise you go up (from back)
    if v[-1] == w[-1]:
      vleft, wleft, scoreleft = hirschberg(v[0:i_star-1], w[0:mid_point-1], delta, mismatch)
      return vleft + v[-1], wleft + w[-1], scoreleft + 1
    else:
      i_star = i_star - 1

  vleft, wleft, scoreleft = hirschberg(v[0:i_star], w[0:mid_point], delta, mismatch)
  vright, wright, scoreright = hirschberg(v[i_star:],w[mid_point:], delta, mismatch)


  return vleft + vright, wleft + wright, scoreleft + scoreright


In [255]:
# start = time.time() # run if you want to test the time complexity
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
mismatch = -1
for key in keys:
    delta[key] = {k : v for (k,v) in zip(keys, [1 if key == key_other  else mismatch for key_other in keys])}

tracemalloc.start()
print(hirschberg("ATCTATCTGGGATCTGGCTGTCTGT", "ATGCGATGCGTATGCGGACTTGACT", delta, mismatch))

print(tracemalloc.get_traced_memory())
tracemalloc.stop()

# end = time.time() # if you want to benchmark time
# print(end - start)



('AT-CTAT-CTGGGAT-CTGG-C-TGTCTGT', 'ATGCGATGC-GT-ATGC-GGACTTGACT--', 4)
(5502, 16179)


In [239]:
# Benchmarking Sequences Used

# 3 ATC ATG
# 4 ATCT ATGC
# 5 ATCTG ATGCG
# 6 ATCTGG ATGCGT
# 7 ATCTGGC ATGCGTG
# 10 ATCTGGCTGT ATGCGTGACT
# 15 GCTATCTGGCTGTGT ATATGCGTGACTGCG
# 20 ATCTGGATCTGGCTGTCTGT ATGCGATGCGTGACTTGACT
# 25 ATCTATCTGGGATCTGGCTGTCTGT ATGCGATGCGTATGCGGACTTGACT
# 30 ATCTATCTGGGATCTGATCTGGCTGTCTGT ATGCGATGCGTATGCGATGCGGACTTGACT
# 35 ATCTATCTGGGATCTGATCTGGATCTGCTGTCTGT ATGCGATGCGTATGCGATGCGATGCGGACTTGACT
# 40 ATCTATCTGGGATCTGATCTGATCTGGATCTGCTGTCTGT ATGCGATGCGTATGCGATCTGATGCGATGCGGACTTGACT
# 50 ATCTATCTGATGCGTGACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATGCGATGCGTATGCGATCTGGCTGTATCTGATGCGATGCGGACTTGACT
# 60 ATCTATCTGATGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATGCGATGCATCTGGCTGTGTATGCGATCTGGCTGTATCTGATGCGATGCGGACTTGACT
# 70 ATCTATCTGAATGCGTGACTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGGCTGTATCTGATGCGATGCGGACTTGACT
# 90 ATCTATCTGAATGCGTGAATGCGATGCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGGCTGTATCTGATGCGATGCGGACTTGACT
# 100 ATCTATCTGAAATGCGTGACTTGCGTGAATGCGATGCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATCTGATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGGCTGTATCTGATGCGATGCGGACTTGACTGCTGT
# 150 ATCTATCTGATGCGTGACTGGATCTGATATCTATCTGAAATGCGTGACTTGCGTGAATGCGATGCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGTCTGATCTGGATCTGCTGTCTGT ATCTGATGCGATGCATCTGGCTGTGTATGATCTGGCTGATGCGATGCGTATGCGATCTGGCTGTATCTGATGCGATGCGGACTTGACTTCGATCTGGCTGTATCTGATGCGATGCGGACTTGACTGCTGT
# 200 ATCTATCTGAAATGCGTGACTTGCGTGAATGCGATGATCTATCTGAAATGCGTGACTTGCGTGAATGCGATGCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGTCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATCTGATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGGCTGTATCTGATGCGATGCGGACTTGACTGCTGTATCTGGCTGTATCTGATGCGATGCGGACTTGACTGCTGT
# 250 ATCTATCTGAAATGCGATCTATCTGATGCGTGACTGGATCTGATCTGATCTGGATCTGCTGTCTGTTGACTTGCGTGAATGCGATGATCTATCTGAAATGCGTGACTTGCGTGAATGCGATGCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGTCGTGACTTGACTCTTGCGTGATGCGTGACTACTGGATCTGATCTGATCTGGATCTGCTGTCTGT ATCTGATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGATGCGATGCATCTGGCTGTGTATGATCTGGCTGTCGATCTGGCTGTATCATGCGATGCGTATGCGATCTGGCTGTATCTGATGCGATGCGGACTTGACTTGATGCGATGCGGACTTGACTGCTGTATCTGGCTGTATCTGATGCGATGCGGACTTGACTGCTGT